# Pytree 사용하기

JAX는 딕셔너리, 배열, 중첩 리스트, 딕셔너리 리스트 혼합 중첩 등 다양한 구조에 대한 연산을 지원하는데, 이러한 구조들을 JAX에서는 pytree라고 부릅니다. 해당 문서에서는 pytree를 잘 사용하는 법과 실수할 수 있는 부분들을 짚고 넘어가겠습니다.

## pytree란

[공식 도큐먼트](https://jax.readthedocs.io/en/latest/working-with-pytrees.html) 에서 pytree에 대해 설명한 글을 직역하면 다음과 같습니다(어떻게 더 잘 번역을 해야할지 감이 안와 공식 도큐먼트를 그대로 인용합니다.):

*파이 트리는 컨테이너 형태의 Python 객체들로 구성된 구조입니다. “리프(leaf)” 파이 트리와 더 많은 파이 트리들로 이루어질 수 있습니다. 파이 트리는 리스트, 튜플, 사전을 포함할 수 있습니다. 리프(leaf)는 배열과 같이 파이 트리가 아닌 모든 것을 의미하지만, 단일 리프도 파이 트리입니다.*

머신러닝의 관점에서 pytree는 다음과 같은 정보를 포함할 수 있습니다:

- 모델 파라메터
- 데이터셋
- 강화학습 문제에서의 에이전트 observation

아래는 간단한 pytree 예제입니다. example_trees 라는 pytree object를 만들어 보겠습니다:

In [1]:
import jax
import jax.numpy as jnp

example_trees = [
    [1, 'a', object()],
    (1, (2, 3), ()),
    [1, {'k1': 2, 'k2': (3, 4)}, 5],
    {'a': 2, 'b': (2, 3)},
    jnp.array([1, 2, 3]),
]

`jax.tree.leaves()`는 리프 노드들을 flatten한 값을 추출합니다. 아래는 `jax.tree.leaves()` 없이 출력한 결과입니다:

In [2]:
for pytree in example_trees:
    print(f"{repr(pytree):<45} has {len(pytree)} leaves: {pytree} leavers: {pytree}")

[1, 'a', <object object at 0x105ddf680>]      has 3 leaves: [1, 'a', <object object at 0x105ddf680>] leavers: [1, 'a', <object object at 0x105ddf680>]
(1, (2, 3), ())                               has 3 leaves: (1, (2, 3), ()) leavers: (1, (2, 3), ())
[1, {'k1': 2, 'k2': (3, 4)}, 5]               has 3 leaves: [1, {'k1': 2, 'k2': (3, 4)}, 5] leavers: [1, {'k1': 2, 'k2': (3, 4)}, 5]
{'a': 2, 'b': (2, 3)}                         has 2 leaves: {'a': 2, 'b': (2, 3)} leavers: {'a': 2, 'b': (2, 3)}
Array([1, 2, 3], dtype=int32)                 has 3 leaves: [1 2 3] leavers: [1 2 3]


아래 코드는 `jax.tree.leaves()`를 사용하여 리프 노드를 출력한 결과입니다:

In [3]:
for pytree in example_trees:
    leaves = jax.tree.leaves(pytree)
    print(f"{repr(pytree):<45} has {len(leaves)} leaves: {leaves} leavers: {leaves}")

[1, 'a', <object object at 0x105ddf680>]      has 3 leaves: [1, 'a', <object object at 0x105ddf680>] leavers: [1, 'a', <object object at 0x105ddf680>]
(1, (2, 3), ())                               has 3 leaves: [1, 2, 3] leavers: [1, 2, 3]
[1, {'k1': 2, 'k2': (3, 4)}, 5]               has 5 leaves: [1, 2, 3, 4, 5] leavers: [1, 2, 3, 4, 5]
{'a': 2, 'b': (2, 3)}                         has 3 leaves: [2, 2, 3] leavers: [2, 2, 3]
Array([1, 2, 3], dtype=int32)                 has 1 leaves: [Array([1, 2, 3], dtype=int32)] leavers: [Array([1, 2, 3], dtype=int32)]


파이썬 컨테이너 형태의 트리 구조로 이루어진 객체들은 모두 pytree로 취급할 수 있습니다. JAX는 pytree 레지스트리를 가지고 있고 레지스트리에 등록된 객체를 pytree로 인식하는데, 리스트, 딕셔너리, 튜플은 기본적으로 pytree 레지스트리에 등록되어 있습니다. 클래스 형태 또한 pytree 레지스트리에 등록한다면 pytree로 인식되게 할 수 있으며, 레지스트리에 등록되지 않은 모든 객체들은 리프 노드로 취급됩니다.

아래 [사용자 정의 pytree](#사용자-정의-pytree) 섹션에서 커스텀 pytree를 만들고 pytree 레지스트리에 등록하는 방법을 알아보겠습니다.

## 대표적인 pytree 함수들

JAX에는 pytree를 다루기 위한 다양한 함수들이 있으며, `jax.tree_util` 아래에 위치하고 있습니다. 하지만 편의를 위해 `jax.tree`로도 접근할 수 있습니다(Aliasing).

### `jax.tree.map`

가장 흔하게 사용되는 pytree 함수는 `jax.tree.map()`입니다. 파이썬 기본함수 `map`과 유사하게 작동하지만 pytree 내부 모든 객체에 대해서 적용됩니다:

In [5]:
list_of_lists = [
    [1, 2, 3],
    [1, 2],
    [1, 2, 3, 4],
]

jax.tree.map(lambda x: x*2, list_of_lists)

[[2, 4, 6], [2, 4], [2, 4, 6, 8]]

## 사용자 정의 pytree